In [1]:
%load_ext autoreload
%autoreload 2
# Combine graph features and use standard algorithms

In [28]:
import numpy as np
import pandas as pd
from utils import run_ML
from sklearn.metrics import f1_score

In [3]:
# !pip install tldextract

In [4]:
data_dir = "data/URLdatasetX2_1.csv"
df = pd.read_csv(data_dir,index_col=0)

In [5]:
df.shape, df.head(2)

((2802, 2),
                                           url        type
 0       http://www.crestonwood.com/router.php  legitimate
 1  http://vamoaestudiarmedicina.blogspot.com/  legitimate)

In [30]:
# smalldata = df.sample(n = 20000, random_state=1)
smalldata = df

In [31]:
if 'PhiUSIIL' in data_dir:
    urldata = np.array(smalldata[['URLLength', 'DomainLength', 'IsDomainIP', \
           'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb',\
           'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation',\
           'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL',\
           'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',\
           'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',\
           'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS']])
    labels = smalldata.iloc[:,-1].values
    run_ML(urldata, labels, "PhiUSIILmock", "standard")
if 'URLdatasetX2' in data_dir:
    labels = smalldata.iloc[:,-1].values
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

### Feature extraction

In [32]:
from utils import extract_features

In [33]:
# Example usage:
url = "http://www.example.com/path/to/==file.html"
url_features = extract_features(url)
print(url_features)

{'domain': 'www.example.com', 'num_subdomains': 2, 'contains_ip': 0, 'path_length': 20, 'num_path_segments': 3, 'uses_https': 0, 'file_extension': 'html', 'count_special_characters': 11, 'count_non_alphanumeric_characters': 11, 'TLD': 'com', 'count_obfuscated_characters': 0, 'letter_ratio_in_url': 0.7380952380952381, 'digit_ratio_in_url': 0.0, 'count_equals_in_url': 2, 'NoOfAmpersandInURL': 0, 'CharContinuationRate': 0.11904761904761904, 'ratio_obfuscated_characters': 0.0, 'NoOfQMarkInURL': 0}


In [34]:
# print(url_features.keys())

In [35]:
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [36]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [37]:
# phish_url_df.head(2)

In [38]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [39]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,11,1,0,47,7,7,38,0,0.810811,0.0,0,0,0.135135,0.0,0
1,2,0,1,1,0,0,6,6,38,0,0.857143,0.0,0,0,0.047619,0.0,0


In [40]:
run_ML(phish_url_df, labels, "URLdatasetX2", "manual")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.86]


In [41]:
## Numerical features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
run_ML(np.array(phish_url), labels, "URLdatasetX2", "manual_numerical")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.83]


In [42]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

In [43]:
# hyperlink_features = np.zeros((smalldata.shape[0], 48))
# enum = 0;
# for url in list(smalldata.iloc[:,0]):
#     # url = 'https://polarklimatsgserver.blogspot.com/'
#     try:    
#         reqs = requests.get(url)
#         soup = BeautifulSoup(reqs.text, 'html.parser')
#         urls = []
#         count = 0;
#         for link in soup.find_all('a'):
#             # print(link.get('href'))
#             weblink = link.get('href')
#             if (weblink is not None) and ('http' in weblink):
#                 urls.append(weblink)
#             count += 1
#             if count > 50:
#                 break
#         # extract numerical features in link
#         if len(urls) > 0:
#             hyperlink_data = []
#             for link in urls:
#                 try:
#                     url_features = extract_numerical_features(link)
#                     datalinkssss = list(url_features.values())
#                 except ValueError as ve:
#                     datalinkssss = list(np.zeros(15))
#                 hyperlink_data.append(datalinkssss)
#             hyperlink_data = np.array(hyperlink_data)
#             hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),
#                                   hyperlink_data.min(axis=0),hyperlink_data.max(axis=0), hyperlink_data.mean(axis=0)))
#         else:
#             hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),np.zeros(45)))
    
#     except ConnectionError as e:
#         print("No rep", end = ',')
#         hyper_np = np.zeros(48)
#     hyperlink_features[enum, :] = hyper_np
#     print(enum, end =',')
#     enum += 1

In [44]:
# for url in list(smalldata.iloc[:,0]):
def get_features(idx):
    url = smalldata.iloc[idx,0]
    # url = 'https://polarklimatsgserver.blogspot.com/'
    try:    
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        urls = []
        count = 0;
        for link in soup.find_all('a'):
            # print(link.get('href'))
            weblink = link.get('href')
            if (weblink is not None) and ('http' in weblink):
                urls.append(weblink)
            count += 1
            if count > 50:
                break
        # extract numerical features in link
        if len(urls) > 0:
            hyperlink_data = []
            for link in urls:
                try:
                    url_features = extract_numerical_features(link)
                    datalinkssss = list(url_features.values())
                except ValueError as ve:
                    datalinkssss = list(np.zeros(15))
                hyperlink_data.append(datalinkssss)
            hyperlink_data = np.array(hyperlink_data)
            hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),
                                  hyperlink_data.min(axis=0),hyperlink_data.max(axis=0), hyperlink_data.mean(axis=0)))
        else:
            hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),np.zeros(45)))
    
    except ConnectionError as e:
        # print("No rep", end = ',')
        hyper_np = np.zeros(48)
    # hyperlink_features[enum, :] = hyper_np
    # print(enum, end =',')
    # enum += 1
    return (idx, hyper_np)

In [45]:
from joblib import Parallel, delayed
results = Parallel(n_jobs=32)(delayed(get_features)(i) for i in range(len(smalldata.index)))

In [46]:
hyperlink_features = np.zeros((smalldata.shape[0], 48))
for idx, hyper_np in results:
    # print(idx, hyper_np)
    hyperlink_features[idx, :] = hyper_np

In [47]:
# np.save('data/hyperlink_features.npy', hyperlink_features) # save
# np.save('data/phish_url_df.npy', phish_url_df.to_numpy()) # save

In [48]:
run_ML(np.concatenate((phish_url_df.to_numpy(), hyperlink_features),axis=1), labels, "URLdatasetX2", "hyperlinks_features")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.81 0.92]


In [49]:
n_samples = len(smalldata.index)
np.random.seed(0)
train_idx = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
test_idx = list(set(list(range(n_samples))).difference(set(train_idx)))
data_df = np.concatenate((phish_url_df.to_numpy(), hyperlink_features),axis=1)
import lightgbm as lgb
model = lgb.LGBMClassifier(verbose=-1)
model.fit(data_df[train_idx], labels[train_idx])
y_predict=model.predict(data_df[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))

0.9031924072476273
